In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from PIL import Image
import open_clip
# import requests
import random
import os
import json
from PIL import Image
import cv2
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import gc
import torch.nn as nn
from typing import List
from numpy import ndarray
import pdb


In [45]:
from data import BuildVQADatasetSingleLanguage, VQADatasetSingleLanguage, MedVQADataset
from CFG_awx import CFG

In [4]:

train_dir = CFG.train_groups

train_img_paths = list(map(lambda x: os.path.join(train_dir, x), os.listdir(train_dir)))
train_dataset_es = BuildVQADatasetSingleLanguage(transforms=CFG.data_transforms['train'], 
                                json_file=CFG.train_json,
                                train_dir=CFG.train_groups,
                                language = 'content_es')


val_dir = CFG.valid_groups

val_img_paths = list(map(lambda x: os.path.join(val_dir, x), os.listdir(val_dir)))
val_dataset_es = BuildVQADatasetSingleLanguage(transforms=CFG.data_transforms['valid'], 
                                json_file=CFG.valid_json,
                                train_dir=CFG.valid_groups, train=False,
                                language='content_es')


test_dir = CFG.test_groups
test_dataset_es = BuildVQADatasetSingleLanguage(transforms=CFG.data_transforms['valid'], 
                                json_file=CFG.test_json,
                                train_dir=CFG.test_groups, train=False,
                                language='content_es')
idx = 42

print(train_dataset_es[idx][3])
# print(f"Question: {val_dataset_es[idx][2]}")
print(len(val_dataset_es), f"Question: {val_dataset_es[idx][2]}")
print(len(test_dataset_es), f"Question: {test_dataset_es[idx][2]}")


['El liquen dorado es un tipo de enfermedad de la piel pigmentada púrpura.', '¿Liquen Dorado Amarillo?', 'Patología.']
56 Question: Esto ocurre todos los veranos, sólo en la parte inferior de la pierna. Desaparece sola en un mes. Pica bastante. Empieza como una pequeña mancha roja, que va creciendo poco a poco.
100 Question: Descripción de la situación: Niña de 2 años.  Tiene dos pequeñas masas en el muslo izquierdo, 
en la zona de pliegue entre la nalga y la parte posterior del muslo.  Las masas son congénitas, pero no se detectaron al nacimiento, 
y crecieron con relativa rapidez antes de cumplir un año.  No se realizó tratamiento. Visualmente, se están extendiendo muy 
lentamente a ambos lados.  No pican ni duelen ni molestan.  Piel suave, sin sensación de hinchazón. 
No alteración cutánea en el lugar de una nueva lesión, ni donde originalmente crecían las masas.  
Hay un poco de escarificación cuando se seca en otoño e invierno. Otras observaciones: Es fácil dejar cicatrices en la 

In [5]:
#  (encounter_id, self.language), images, question, answers

In [5]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16')
model.load_state_dict(torch.load('pathgenclip.pt'))
print("All keys matched succesfully")
model.eval()
print("PathGen loaded succesfully")

All keys matched succesfully
PathGen loaded succesfully


In [6]:
image = train_dataset_es[0][1]
image = image.permute(0, 3, 1, 2)
image.shape


torch.Size([4, 3, 224, 224])

In [7]:
def extract_image_features(dataset, model, split, feature_dir, recompute=False):
    """
    Extract image features using a pre-trained model and save them to disk,
    with an option to recompute existing features.
    
    Args:
        dataset (torch.utils.data.Dataset): The dataset containing the images.
        model (torch.nn.Module): The pre-trained model used for feature extraction.
        split (str): The dataset split ('train', 'valid', or 'test').
        recompute (bool): If True, recompute features even if files already exist.
    """
    # feature_dir = "Features/pathgen"
    os.makedirs(f'{feature_dir}/{split}', exist_ok=True)
    
    for i in range(len(dataset)):
        img_id = dataset[i][0][0]
        output_file = os.path.join(feature_dir, split, f'{img_id}.pt')
        
        # Check if the file already exists and recompute flag
        if recompute or not os.path.exists(output_file):
            with torch.no_grad(), torch.amp.autocast(device_type='cuda'):
                image = dataset[i][1]
                image = image.permute(0, 3, 1, 2)
                image_features = model.encode_image(image)
                torch.save(image_features, output_file)

In [8]:
feature_extractor = 'pathgen'
feature_dir = os.path.join('Features', feature_extractor)
extract_image_features(dataset=val_dataset_es, model=model, split='valid', feature_dir=feature_dir, recompute=False)

In [9]:
# feature_extractor = 'pathgen'
# feature_dir = os.path.join('Features', feature_extractor)
# extract_image_features(dataset=test_dataset_es, model=model, split='test', feature_dir=feature_dir, recompute=False)

In [10]:
# idx = 0
# img_ids = []
# img_paths = []
# all_questions = []
# all_answers = []
# img_dict = {} 
# for idx in tqdm(range(len(train_dataset_es))):
#     d = train_dataset_es[idx]
#     img_id = d[0][0]
#     filename = f'{img_id}.jpg'
#     prefix_i = model.encode_image(d[1].permute(0, 3, 1, 2)).cpu()  
#     question = d[2]
#     answer = d[3]
#     if img_id not in img_dict.keys():
#         img_dict[img_id] = [[question],answer,prefix_i,filename]
#     else:
#         img_dict[img_id][0].append(question)
#         img_dict[img_id][1].append(answer)

In [11]:
# all_img_prefixes = []

# for img_id, imgs in enumerate(img_dict.keys()):
#     all_img_prefixes.append(img_dict[imgs][2])
#     all_questions.append(img_dict[imgs][0])
    
        
#     all_answers.append(img_dict[imgs][1])
#     img_ids.append(img_id)
#     img_paths.append(img_dict[imgs][2])

# all_data = {"img_prefix": torch.cat(all_img_prefixes, dim=0), "img_ids": img_ids, "questions": all_questions,'answers': all_answers,'img_path': img_paths}


In [59]:
trainset = MedVQADataset(data=train_dataset_es)

In [60]:
trainset[42][1]

tensor([   79.,  2301., 44424.,  ...,     0.,     0.,     0.])

In [61]:
trainset[0]

(tensor([[[[0.9904, 0.7826, 0.7236],
           [0.9907, 0.7874, 0.7319],
           [0.9938, 0.8013, 0.7540],
           ...,
           [0.6743, 0.4870, 0.3756],
           [0.6784, 0.4927, 0.3796],
           [0.6742, 0.4891, 0.3742]],
 
          [[0.9866, 0.7769, 0.7173],
           [0.9869, 0.7832, 0.7274],
           [0.9885, 0.7966, 0.7479],
           ...,
           [0.6713, 0.4848, 0.3721],
           [0.6759, 0.4906, 0.3768],
           [0.6712, 0.4858, 0.3710]],
 
          [[0.9874, 0.7733, 0.7145],
           [0.9858, 0.7770, 0.7213],
           [0.9867, 0.7950, 0.7448],
           ...,
           [0.6726, 0.4821, 0.3705],
           [0.6741, 0.4847, 0.3724],
           [0.6691, 0.4791, 0.3655]],
 
          ...,
 
          [[0.9410, 0.7683, 0.6703],
           [0.9391, 0.7664, 0.6685],
           [0.9401, 0.7653, 0.6673],
           ...,
           [0.0351, 0.0278, 0.0319],
           [0.0311, 0.0242, 0.0281],
           [0.0290, 0.0227, 0.0264]],
 
          [[0.9312,

In [67]:
trainset[0][1][0:10]

tensor([7.9000e+01, 2.3010e+03, 4.4424e+04, 2.5000e+01, 2.2000e+02, 3.1180e+03,
        2.3503e+04, 1.1530e+03, 6.8000e+01, 3.6900e+02])

In [50]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [52]:
tokenizer.decode(trainset[0][1])

'pregunta: Un paciente con derrame pleural está acompañado de una erupción sistémica, como se ve en la imagen (actualmente solo está disponible la imagen de la espalda). contexto:""respuesta [\'La psoriasis parece no tener relación con el derrame pleural.\', \'Psoriasis Típica\', \'Psoriasis\', \'Psoriasis Típica\']<|endoftext|>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [33]:
q_lens = []
a_lens = []
for i in range(len(test_dataset_es)):
    question = test_dataset_es[i][2]
    q_lens.append(len(tokenizer.encode(question)))
# for answer in data_train['answers']:
#     a_lens.append(len(tokenizer.encode(str(answer))))

In [40]:
max_track = 0
for i in range(len(train_dataset_es)):
    answer = train_dataset_es[i][3]
    tokenized_answers = tokenizer.batch_encode_plus(answer)['input_ids']
    curr_max = max([len(ans) for ans in tokenized_answers])
    if curr_max > max_track:
        max_track = curr_max

In [41]:
max_track

870

In [42]:
int(np.mean(q_lens)+2*np.std(q_lens)) # TRAIN: 388, max is 635
np.max(q_lens)

635

In [34]:
int(np.mean(q_lens)+2*np.std(q_lens)) # TEST: 343

343